In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load CIFAR-10 dataset as NumPy arrays
(x_train, y_train), (x_test, y_test) = tfds.as_numpy(tfds.load(
    'cifar10',
    split=['train', 'test'],
    batch_size=-1,
    as_supervised=True,
))

# Normalize the images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\shaif\tensorflow_datasets\cifar10\3.0.2.incomplete3F5GNL\cifar10-train.tfrecord*...:   0%| …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\shaif\tensorflow_datasets\cifar10\3.0.2.incomplete3F5GNL\cifar10-test.tfrecord*...:   0%|  …

Dataset cifar10 downloaded and prepared to C:\Users\shaif\tensorflow_datasets\cifar10\3.0.2. Subsequent calls will reuse this data.


In [8]:
def get_augmenter():
    return tf.keras.Sequential([
        layers.RandomFlip('horizontal'),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.3),
        layers.RandomContrast(0.1),
    ])

def generate_positive_pairs(images):
    augmenter = get_augmenter()
    augmented_images_1 = augmenter(images)
    augmented_images_2 = augmenter(images)
    return augmented_images_1, augmented_images_2

# Create a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(256)

# Apply augmentations to generate positive pairs
train_dataset = train_dataset.map(lambda x: generate_positive_pairs(x), 
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)


In [ ]:
def create_simclr_model(input_shape=(32, 32, 3)):
    # Input layers for the two augmented images
    inputs_1 = layers.Input(shape=input_shape)
    inputs_2 = layers.Input(shape=input_shape)

    # Shared base model with ResNet50
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    # Pass both inputs through the same base model
    features_1 = base_model(inputs_1)
    features_2 = base_model(inputs_2)

    # Global average pooling
    pooled_features_1 = layers.GlobalAveragePooling2D()(features_1)
    pooled_features_2 = layers.GlobalAveragePooling2D()(features_2)

    # Projection head
    projection_1 = layers.Dense(128, activation='relu')(pooled_features_1)
    projection_1 = layers.Dense(128)(projection_1)
    
    projection_2 = layers.Dense(128, activation='relu')(pooled_features_2)
    projection_2 = layers.Dense(128)(projection_2)

    # Final SimCLR model
    simclr_model = Model(inputs=[inputs_1, inputs_2], outputs=[projection_1, projection_2])

    return simclr_model

# Create the SimCLR model
simclr_model = create_simclr_model()
simclr_model.summary()


In [7]:
def contrastive_loss(projections, temperature=0.1):
    # Ensure the projections tensor is in float32 (to prevent type issues)
    projections = tf.cast(projections, tf.float32)
    
    # Normalize the projections
    projections = tf.math.l2_normalize(projections, axis=1)
    
    # Calculate similarity matrix
    similarity_matrix = tf.matmul(projections, projections, transpose_b=True)
    
    # Temperature scaling - make sure temperature is broadcasted
    logits = similarity_matrix / tf.cast(temperature, tf.float32)
    
    # Create labels (diagonal is positive, self-similarity)
    labels = tf.eye(tf.shape(logits)[0])
    
    # Compute contrastive loss
    loss = tf.nn.softmax_cross_entropy_with_logits(labels, logits)
    return tf.reduce_mean(loss)

# Compile the model
simclr_model.compile(optimizer=tf.keras.optimizers.Adam(), loss=contrastive_loss)

# Train the model
simclr_model.fit(train_dataset, epochs=2)


Epoch 1/2


InvalidArgumentError: Graph execution error:

Detected at node 'contrastive_loss/truediv' defined at (most recent call last):
    File "C:\Users\shaif\anaconda3\envs\tens\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\shaif\anaconda3\envs\tens\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\ipykernel\kernelapp.py", line 728, in start
      self.io_loop.start()
    File "C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\shaif\anaconda3\envs\tens\lib\as